# 2.1 Modelo Linear + Features.ipynb — Modelo Preditivo Linear com engenharia de Features (Regressão Linear)

Este notebook implementa uma *segunda versão* de um modelo preditivo para estimar a **demanda horária** do pronto-socorro.

O objetivo desta segunda versão é manter a mesma metologia utilizada no primeiro modelo, porém, aprimorar o modelo linear simples utilizando engenharia de features mais robusta, utilizando:

- Lags de demanda (janelas temporais)
- Médias móveis (rolling windows)

Seguiremos avaliando estas etapas:
1. Seleção das features e preparação dos dados
2. Divisão treino/teste respeitando series temporais
3. Treinamento do modelo baseline (Regressão Linear)
4. Predição
5. Avaliação com MAE, RMSE, MAPE e R²
6. Gráficos de diagnóstico

---

# 1. Importar bibliotecas e configurações iniciais

### 1.0 Instalações

In [1]:
# ! pip install scikit-learn

### 1.1 Importações

In [2]:
import pandas as pd
import numpy as np
import altair as alt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score, root_mean_squared_error, mean_squared_error

import locale

In [3]:
import sys
sys.path.append("../src")

from features.feature_engineering import create_lag_features, create_rolling_features, add_time_features

### 1.2 Configurações de bibliotecas

In [4]:
# Desabilitar a limitação de linhas em gráficos do Altair
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [5]:
locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')

'pt_BR.UTF-8'

# 2. Importar e tratar dados

### 2.1 Importando dados

In [6]:
CAMINHO_DADOS = '../data/raw/dataset_pronto_socorro.csv'

df = pd.read_csv(CAMINHO_DADOS)

In [7]:
df.head()

,datetime,day_of_week,month,is_weekend,temperature,rain_mm,demand
0,2023-01-01 00:00:00,6,1,1,24.483571,0.353269,29.0
1,2023-01-01 01:00:00,6,1,1,21.308678,5.847757,30.0
2,2023-01-01 02:00:00,6,1,1,25.238443,1.141991,30.0
3,2023-01-01 03:00:00,6,1,1,29.615149,0.524987,33.0
4,2023-01-01 04:00:00,6,1,1,20.829233,0.820610,33.0


### 2.2 Tratamento de Dados

In [8]:
df.drop(columns=['day_of_week'], inplace=True)

In [9]:
# Converter coluna de data/hora para datetime e definir índice
if 'datetime' in df.columns:
    df['datetime'] = pd.to_datetime(df['datetime'])
    df = df.set_index('datetime')

### 2.3 Engenharia de Features

In [10]:
# Adicionar variáveis temporais
df = add_time_features(df)

# Lags essenciais: 1 hora, 24h, 48h, 1 semana (168h)
df = create_lag_features(df, lags=[1, 2, 3, 24, 48, 168])

# Rolling windows
df = create_rolling_features(df, windows=[3, 6, 12, 24])

# Remover linhas com NaNs causados pelos lags/rolling
df = df.dropna()

In [11]:
df.head()

,month,is_weekend,temperature,rain_mm,demand,hour,dayofweek,demand_lag_1,demand_lag_2,demand_lag_3,...,demand_lag_48,demand_lag_168,demand_roll_mean_3,demand_roll_std_3,demand_roll_mean_6,demand_roll_std_6,demand_roll_mean_12,demand_roll_std_12,demand_roll_mean_24,demand_roll_std_24
datetime,,,,,,,,,,,,,,,,,,,,,
2023-01-08 00:00:00,1,1,20.773059,2.344874,25.0,0,6,28.0,26.0,20.0,...,22.0,29.0,26.333333,1.527525,24.166667,3.060501,24.083333,3.704011,25.583333,3.877658
2023-01-08 01:00:00,1,1,18.231319,0.268283,25.0,1,6,25.0,28.0,26.0,...,27.0,30.0,26.000000,1.732051,24.833333,2.639444,23.833333,3.511885,25.375000,3.762660
2023-01-08 02:00:00,1,1,17.552428,0.192845,22.0,2,6,25.0,25.0,28.0,...,24.0,30.0,24.000000,1.732051,24.333333,2.875181,23.583333,3.528026,25.000000,3.623594
2023-01-08 03:00:00,1,1,17.920949,1.191458,34.0,3,6,22.0,25.0,25.0,...,26.0,33.0,27.000000,6.244998,26.666667,4.082483,24.000000,4.410731,25.250000,4.024382
2023-01-08 04:00:00,1,1,21.614491,3.771095,35.0,4,6,34.0,22.0,25.0,...,23.0,33.0,30.333333,7.234178,28.166667,5.269409,24.833333,5.441145,25.666667,4.488310


# 3 Desenvolvimento de Modelo Preditivo

### 3.1 Seleção das variáveis do modelo

In [12]:
features = [
    # variáveis temporais
    "hour", "dayofweek", "month", "is_weekend",

    # lags
    "demand_lag_1", "demand_lag_2", "demand_lag_3",
    "demand_lag_24", "demand_lag_48", "demand_lag_168",

    # rolling windows
    "demand_roll_mean_3", "demand_roll_mean_6",
    "demand_roll_mean_12", "demand_roll_mean_24",
    "demand_roll_std_3",  "demand_roll_std_6",
    "demand_roll_std_12", "demand_roll_std_24",
]

target = 'demand'


In [13]:
X = df[features]
y = df[target]

### 3.2 Divisão dos dados em treino/teste

In [14]:
# Neste caso, está sendo utilizada uma separação simples, onde 80% dos dados são usados para treino e 20% para teste.
split_ratio = 0.8
split_point = int(len(df) * split_ratio)

In [15]:
# Divindo os dados em treino e teste
X_train, X_test = X.iloc[:split_point], X.iloc[split_point:]
y_train, y_test = y.iloc[:split_point], y.iloc[split_point:]

In [16]:
print(f"Registros treino: {locale.format_string('%d',len(X_train),grouping=True)}")
print(f"Registros teste:  {locale.format_string('%d',len(X_test),grouping=True)}")

Registros treino: 13.882
Registros teste:  3.471


### 3.3 Treinar Regressão Linear

In [17]:
model = LinearRegression()
model.fit(X_train, y_train)

,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [18]:
print("Intercepto:", model.intercept_)
print("Coeficientes:")
for feat, coef in zip(features, model.coef_):
    print(f"{feat}: {coef:.4f}")

Intercepto: 2.1316282072803006e-14
Coeficientes:
hour: 0.0000
dayofweek: 0.0000
month: -0.0000
is_weekend: 0.0000
demand_lag_1: -1.0000
demand_lag_2: -1.0000
demand_lag_3: 0.0000
demand_lag_24: 0.0000
demand_lag_48: 0.0000
demand_lag_168: 0.0000
demand_roll_mean_3: 3.0000
demand_roll_mean_6: -0.0000
demand_roll_mean_12: -0.0000
demand_roll_mean_24: 0.0000
demand_roll_std_3: -0.0000
demand_roll_std_6: 0.0000
demand_roll_std_12: -0.0000
demand_roll_std_24: -0.0000


### 3.4 Predições

In [19]:
y_pred = model.predict(X_test)

# 4. Análise detalhada do resultado

### 4.1 Criar dataframe com resultados

In [20]:
df_eval = pd.DataFrame({
    "y_true": y_test,
    "y_pred": y_pred
})

In [21]:
# Erro residual
# Residual positivo = modelo subestimou
# Residual negativo = modelo superestimou

df_eval["residual"] = df_eval["y_true"] - df_eval["y_pred"]

In [22]:
df_eval.head()

,y_true,y_pred,residual
datetime,,,
2024-08-08 10:00:00,18.0,18.0,-7.105427e-15
2024-08-08 11:00:00,18.0,18.0,-7.105427e-15
2024-08-08 12:00:00,16.0,16.0,-7.105427e-15
2024-08-08 13:00:00,12.0,12.0,-8.881784e-15
2024-08-08 14:00:00,14.0,14.0,-1.243450e-14


## 4.2 Métricas de Avaliação

In [23]:
mae = mean_absolute_error(df_eval['y_true'], df_eval['y_pred'])
print("MAE:", mae)

MAE: 9.561160084153034e-15


In [24]:
# RMSE penaliza erros grandes → ótimo para detectar picos que o modelo perdeu.
rmse = root_mean_squared_error(df_eval['y_true'], df_eval['y_pred'])
print("RMSE:", rmse)

RMSE: 1.1265989309538024e-14


In [25]:
# MAPE ajuda a entender o erro percentual, mas pode distorcer quando valores são pequenos.
mape = np.mean(np.abs(df_eval["residual"] / df_eval["y_true"])) * 100
print(f"MAPE: {mape:.2f}%")

MAPE: 0.00%


In [26]:
r2 = r2_score(df_eval['y_true'], df_eval['y_pred'])
print("R²:", r2)

R²: 1.0


## 4.3 Distribuição dos Resíduos

In [27]:
# Se os resíduos não forem centrados em 0 → viés no modelo.
# Caudas pesadas → eventos extremos não capturados.

In [28]:
# Histograma
hist =alt.Chart(df_eval.reset_index()).mark_bar().encode(
    x=alt.X("residual", bin=alt.Bin(maxbins=50), title='Distribuição dos Resíduos'),
    y=alt.Y('count()',title='Frequência')
)
# Linha vertical no zero
linha_zero = (
    alt.Chart(pd.DataFrame({"x": [0]}))
    .mark_rule(color="black")
    .encode(x="x:Q")
)
# KDE (curva suavizada) — opcional
kde = (
    alt.Chart(df_eval.reset_index())
    .transform_density(
        "residual",
        as_=["residual", "density"]
    )
    .mark_line(color="red")
    .encode(
        x="residual:Q",
        y="density:Q"
    )
)

(hist + linha_zero + kde).resolve_scale(
    y="independent"
).properties(
    width=600,
    height=300,
    title="Distribuição dos resíduos"
)


alt.LayerChart(...)

### 4.4 Error por Hora do dia

- Este gráfico revela se o modelo falha no começo, meio ou final do dia.
- Se os erros forem sistematicamente altos em certas horas → falta de features temporais.

In [30]:
df_hour = df_eval.copy()
df_hour["hour"] = X_test["hour"]

hour_mae = df_hour.groupby("hour",as_index=False)["residual"].apply(lambda x: np.mean(np.abs(x)))
hour_mae.head()

,hour,residual
0,0,1.081740e-14
1,1,1.007836e-14
2,2,1.092953e-14
3,3,1.092953e-14
4,4,1.016471e-14


In [31]:
chart = (
    alt.Chart(hour_mae)
    .mark_bar()
    .encode(
        x=alt.X("hour:O", title="Hora do dia"),
        y=alt.Y("residual:Q", title="Erro absoluto médio (MAE)"),
    )
    .properties(
        width=600,
        height=300,
        title="MAE por hora do dia"
    )
)

chart

alt.Chart(...)

### 4.5 Erro por dia da Semana

- A demanda hospitalar costuma variar muito entre domingo e segunda.
- Se o baseline não captura → será necessário adicionar lags.

In [32]:
df_dow = df_eval.copy()
df_dow["dayofweek"] = X_test["dayofweek"]

dow_mae = df_dow.groupby("dayofweek",as_index=False)["residual"].apply(lambda x: np.mean(np.abs(x)))
dow_mae.head()

,dayofweek,residual
0,0,9.551443e-15
1,1,9.710874e-15
2,2,8.298917e-15
3,3,9.586574e-15
4,4,1.123088e-14


In [33]:
chart = (
    alt.Chart(dow_mae)
    .mark_bar()
    .encode(
        x=alt.X("dayofweek:O", title="Dia da Semana"),
        y=alt.Y("residual:Q", title="Erro absoluto médio (MAE)"),
    )
    .properties(
        width=600,
        height=300,
        title="MAE por dia da semana"
    )
)

chart

alt.Chart(...)

### 4.6 Comparação Real vs. Predição agregado por dia

- Linear Regression pode acertar nível médio mas errar amplitude.
- Se o modelo suaviza demais → pode exigir modelos não lineares.

In [34]:
df_daily = df_eval[["y_true", "y_pred"]].resample("D").sum().reset_index()
df_daily.head()

,datetime,y_true,y_pred
0,2024-08-08,205.0,205.0
1,2024-08-09,381.0,381.0
2,2024-08-10,518.0,518.0
3,2024-08-11,523.0,523.0
4,2024-08-12,386.0,386.0


In [35]:
# Converte para formato long (necessário para múltiplas linhas no Altair)
df_long = df_daily.melt(id_vars="datetime", value_vars=["y_true", "y_pred"],
                       var_name="tipo", value_name="valor")
df_long.head()

,datetime,tipo,valor
0,2024-08-08,y_true,205.0
1,2024-08-09,y_true,381.0
2,2024-08-10,y_true,518.0
3,2024-08-11,y_true,523.0
4,2024-08-12,y_true,386.0


In [40]:
chart = (
    alt.Chart(df_long)
    .mark_line()
    .encode(
        x=alt.X("datetime:T", title="Data"),
        y=alt.Y("valor:Q", title="Demanda"),
        color=alt.Color("tipo:N", title="Série", scale=alt.Scale(
            domain=["y_true", "y_pred"],
            range=["black", "steelblue"]
        )),
        tooltip=["data:T", "tipo:N", "valor:Q"]
    )
    .properties(
        width=1200,
        height=500,
        title="Demanda diária — Real vs Prevista"
    )
)

In [41]:

chart

alt.Chart(...)

### 4.7 Resíduos ao longo de tempo

- Ver se há períodos em que o modelo erra sistematicamente (viés temporal).
- Ver se há heterocedasticidade (erro aumenta em períodos de pico).

In [37]:
# Linha dos resíduos
residual_line = (
    alt.Chart(df_eval.reset_index())
    .mark_line()
    .encode(
        x=alt.X("datetime:T", title="Data"),
        y=alt.Y("residual:Q", title="Resíduo")
    )
)

# Linha horizontal em zero
linha_zero = (
    alt.Chart(pd.DataFrame({"y": [0]}))
    .mark_rule(color="black")
    .encode(y="y:Q")
)

chart = (
    (residual_line + linha_zero)
    .properties(
        width=1200,
        height=400,
        title="Resíduos ao longo do tempo"
    )
)

chart

alt.LayerChart(...)

### 4.8 Interpretação dos coeficientes do modelo

- coef > 0 → aumenta demanda
- coef < 0 → reduz demanda
  
- Permite validar coerência com conhecimento hospitalar.

In [38]:
coef_df = pd.DataFrame({
    "feature": features,
    "coef": model.coef_
})

coef_df.sort_values("coef", ascending=False)

,feature,coef
10,demand_roll_mean_3,3.000000e+00
1,dayofweek,1.221245e-15
13,demand_roll_mean_24,7.974597e-16
3,is_weekend,5.602107e-16
8,demand_lag_48,5.150095e-16
15,demand_roll_std_6,3.519182e-16
0,hour,2.941157e-16
7,demand_lag_24,2.412979e-16
9,demand_lag_168,1.221819e-16
6,demand_lag_3,4.300260e-17


### 4.9 Importância padronizada (coef*std)

- Isso mostra quais variáveis mais impactam a previsão na prática.
- Muito útil para justificar a evolução do modelo.

In [39]:
stds = X_train.std()

coef_imp_df = pd.DataFrame({
    "feature": features,
    "coef": model.coef_,
    "std": stds,
})

coef_imp_df["importance_std"] = coef_imp_df["coef"] * coef_imp_df["std"]
coef_imp_df.sort_values("importance_std", ascending=False)

,feature,coef,std,importance_std
demand_roll_mean_3,demand_roll_mean_3,3.000000e+00,4.476043,1.342813e+01
demand_roll_mean_24,demand_roll_mean_24,7.974597e-16,3.458063,2.757666e-15
demand_lag_48,demand_lag_48,5.150095e-16,5.127961,2.640949e-15
dayofweek,dayofweek,1.221245e-15,2.002369,2.445384e-15
hour,hour,2.941157e-16,6.922920,2.036139e-15
demand_lag_24,demand_lag_24,2.412979e-16,5.132669,1.238502e-15
demand_lag_168,demand_lag_168,1.221819e-16,5.117240,6.252344e-16
demand_roll_std_6,demand_roll_std_6,3.519182e-16,1.008504,3.549108e-16
is_weekend,is_weekend,5.602107e-16,0.451555,2.529660e-16
demand_lag_3,demand_lag_3,4.300260e-17,5.134898,2.208140e-16


# 5. Conclusão

- Podemos notar uma melhora expressiva no resultado no mesmo modelo, apenas utilizando novas variáveis que trazem informações sobre o histórico da série histórica.
- Todas as métricas de dispersão avaliadas chegaram a praticamente zero, mostrando robustez do modelo a ser utilizado neste conjunto de dados.
- **Ponto de Atenção**: Temos que a fonte de dados é sintética, portanto, não reflete 100% a realidade de demandas de um pronto atendimento.

**Próximos passos**
- Utilizar metodologias mais robustas na divisão dos dados, para nos auxiliar na validação do modelo em diferentes janelas de dados.